In [1]:
# Importer les bibliothèques nécessaires
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [14]:
# Télécharger le lexique VADER pour l'analyse de sentiment s'il n'est pas déjà présent
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Nouhaila
[nltk_data]     Chg\AppData\Roaming\nltk_data...


True

In [10]:
# Définir une fonction pour récupérer les données depuis une base de données SQL via une requête SQL
def fetch_data_from_sql():
    # Définir la chaîne de connexion avec les paramètres pour se connecter à la base de données
    conn_str = (
        "Driver={SQL Server};"  # Spécifie le driver pour SQL Server
        "Server=inspiron;"  # Spécifie l'instance SQL Server
        "Database=Marketing;"  # Spécifie le nom de la base de données
        "Trusted_Connection=yes;"  # Utilise l'authentification Windows pour la connexion
    )
    # Établir la connexion à la base de données
    conn = pyodbc.connect(conn_str)
    
    # Définir la requête SQL pour récupérer les données des avis clients
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"
    
    # Exécuter la requête et récupérer les données dans un DataFrame
    df = pd.read_sql(query, conn)
    
    # Fermer la connexion pour libérer les ressources
    conn.close()
    
    # Retourner les données récupérées sous forme de DataFrame
    return df

In [11]:
# Récupérer les données des avis clients depuis la base de données SQL
customer_reviews_df = fetch_data_from_sql()

C:\Users\Nouhaila Chg\AppData\Local\Temp\ipykernel_33400\4265116639.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [15]:
# Initialiser l'analyseur de sentiment VADER pour analyser les sentiments des données textuelles
sia = SentimentIntensityAnalyzer()

In [16]:
# Définir une fonction pour calculer les scores de sentiment à l'aide de VADER
def calculate_sentiment(review):
    # Obtenir les scores de sentiment pour le texte de l'avis
    sentiment = sia.polarity_scores(review)
    # Retourner le score composite, qui est un score normalisé entre -1 (le plus négatif) et 1 (le plus positif)
    return sentiment['compound']

In [17]:
# Définir une fonction pour catégoriser les sentiments en utilisant à la fois le score de sentiment et la note de l'avis
def categorize_sentiment(score, rating):
    # Utiliser à la fois le score de sentiment du texte et la note numérique pour déterminer la catégorie de sentiment
    if score > 0.05:  # Score de sentiment positif
        if rating >= 4:
            return 'Positive'  # Haute note et sentiment positif
        elif rating == 3:
            return 'Mixed Positive'  # Note neutre mais sentiment positif
        else:
            return 'Mixed Negative'  # Faible note mais sentiment positif
    elif score < -0.05:  # Score de sentiment négatif
        if rating <= 2:
            return 'Negative'  # Faible note et sentiment négatif
        elif rating == 3:
            return 'Mixed Negative'  # Note neutre mais sentiment négatif
        else:
            return 'Mixed Positive'  # Haute note mais sentiment négatif
    else:  # Score de sentiment neutre
        if rating >= 4:
            return 'Positive'  # Haute note avec sentiment neutre
        elif rating <= 2:
            return 'Negative'  # Faible note avec sentiment neutre
        else:
            return 'Neutral'  # Note neutre et sentiment neutre

In [18]:
# Définir une fonction pour regrouper les scores de sentiment en plages de texte
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Sentiment fortement positif
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Sentiment légèrement positif
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Sentiment légèrement négatif
    else:
        return '-1.0 to -0.5'  # Sentiment fortement négatif

In [19]:
# Appliquer l'analyse de sentiment pour calculer les scores de sentiment pour chaque avis
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [20]:
# Appliquer la catégorisation des sentiments en utilisant à la fois le texte et la note
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [21]:
# Appliquer le regroupement des sentiments pour catégoriser les scores dans des plages définies
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [23]:
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


In [24]:
# Enregistrer le DataFrame dans un fichier CSV
customer_reviews_df.to_csv('fact_customerReviews_with_sentiment.csv', index=False)